In [ ]:
#In this notebook I play with a notebook i found for using Naive Bayes parameters for weights in Embedding
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
from keras.layers.core import Activation
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, dot
from keras import backend as K
from keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
import tensorflow as tf


In [ ]:
imdb_data = tf.keras.datasets.imdb.load_data(path='imdb.npz', )

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
word_to_index = tf.keras.datasets.imdb.get_word_index(path='imdb.npz')
index_to_word = { y:x for x,y in word_to_index.items()}

In [ ]:
print( len(word_to_index) )
print( len(index_to_word) )

88584
88584


In [ ]:
print( list(word_to_index.items() )[0:10] )
print( list(index_to_word.items() )[0:10] )

[('fawn', 34701), ('tsukino', 52006), ('nunnery', 52007), ('sonja', 16816), ('vani', 63951), ('woods', 1408), ('spiders', 16115), ('hanging', 2345), ('woody', 2289), ('trawling', 52008)]
[(34701, 'fawn'), (52006, 'tsukino'), (52007, 'nunnery'), (16816, 'sonja'), (63951, 'vani'), (1408, 'woods'), (16115, 'spiders'), (2345, 'hanging'), (2289, 'woody'), (52008, 'trawling')]


In [ ]:
(trainx,y_train),(testx,y_test) = imdb_data

In [ ]:
" ".join([str(x) for x in trainx[0] ] )

'1 14 22 16 43 530 973 1622 1385 65 458 4468 66 3941 4 173 36 256 5 25 100 43 838 112 50 670 22665 9 35 480 284 5 150 4 172 112 167 21631 336 385 39 4 172 4536 1111 17 546 38 13 447 4 192 50 16 6 147 2025 19 14 22 4 1920 4613 469 4 22 71 87 12 16 43 530 38 76 15 13 1247 4 22 17 515 17 12 16 626 18 19193 5 62 386 12 8 316 8 106 5 4 2223 5244 16 480 66 3785 33 4 130 12 16 38 619 5 25 124 51 36 135 48 25 1415 33 6 22 12 215 28 77 52 5 14 407 16 82 10311 8 4 107 117 5952 15 256 4 31050 7 3766 5 723 36 71 43 530 476 26 400 317 46 7 4 12118 1029 13 104 88 4 381 15 297 98 32 2071 56 26 141 6 194 7486 18 4 226 22 21 134 476 26 480 5 144 30 5535 18 51 36 28 224 92 25 104 4 226 65 16 38 1334 88 12 16 283 5 16 4472 113 103 32 15 16 5345 19 178 32'

In [ ]:
 veczr =  CountVectorizer(ngram_range=(1,3), binary=True,   token_pattern=r'\w+', max_features=len(index_to_word))

In [ ]:
#join indices as a sentence 
train_data = [ " ".join([str(y) for y in x]) for x in trainx]
test_data = [ " ".join([str(y) for y in x]) for x in testx]

dtm_train  = veczr.fit_transform(train_data)
dtm_test   = veczr.transform(test_data)


In [ ]:
print(train_data[0])
print(dtm_train[0].indices)

1 14 22 16 43 530 973 1622 1385 65 458 4468 66 3941 4 173 36 256 5 25 100 43 838 112 50 670 22665 9 35 480 284 5 150 4 172 112 167 21631 336 385 39 4 172 4536 1111 17 546 38 13 447 4 192 50 16 6 147 2025 19 14 22 4 1920 4613 469 4 22 71 87 12 16 43 530 38 76 15 13 1247 4 22 17 515 17 12 16 626 18 19193 5 62 386 12 8 316 8 106 5 4 2223 5244 16 480 66 3785 33 4 130 12 16 38 619 5 25 124 51 36 135 48 25 1415 33 6 22 12 215 28 77 52 5 14 407 16 82 10311 8 4 107 117 5952 15 256 4 31050 7 3766 5 723 36 71 43 530 476 26 400 317 46 7 4 12118 1029 13 104 88 4 381 15 297 98 32 2071 56 26 141 6 194 7486 18 4 226 22 21 134 476 26 480 5 144 30 5535 18 51 36 28 224 92 25 104 4 226 65 16 38 1334 88 12 16 283 5 16 4472 113 103 32 15 16 5345 19 178 32
[    0 15000 30241 20726 53932 62275 87848 21873 14715 70354 55709 55048
 70824 46855 47086 23797 44161 34875 58030 33982  1390 81707  5914 60351
 71449 83893 43328 57145 37891 18919 23687 22585 42455 46243 46460 55560
  5781 22969 62941 45679 11611 55050

In [ ]:
print("document-term matrix shape (training): (%s, %s)" % (dtm_train.shape))
print("document-term matrix shape (test): (%s, %s)" % (dtm_train.shape))
num_words = len([v for k,v in veczr.vocabulary_.items()]) + 1 # add 1 for 0 padding
print('vocab size:%s' % (num_words))

document-term matrix shape (training): (25000, 88584)
document-term matrix shape (test): (25000, 88584)
vocab size:88585


In [ ]:
def dtm2wid(dtm, maxlen=2000):
    x = []
    nwds = []
    for idx, row in enumerate(dtm):
        seq = []
        indices = (row.indices + 1).astype(np.int64)
        np.append(nwds, len(indices))
        data = (row.data).astype(np.int64)
        count_dict = dict(zip(indices, data))
        for k,v in count_dict.items():
            seq.extend([k]*v)
        num_words = len(seq)
        nwds.append(num_words)
        # pad up to maxlen
        if num_words < maxlen: 
            seq = np.pad(seq, (maxlen - num_words, 0), mode='constant')
        # truncate down to maxlen
        else:                  
            seq = seq[-maxlen:]
        x.append(seq)
    nwds = np.array(nwds)
    print('sequence stats: avg:%s, max:%s, min:%s' % (nwds.mean(), nwds.max(), nwds.min()) )
    return np.array(x)

In [ ]:

maxlen = 2000
x_train = dtm2wid(dtm_train, maxlen=maxlen)
x_test = dtm2wid(dtm_test, maxlen=maxlen)

sequence stats: avg:309.9618, max:1670, min:16
sequence stats: avg:300.58584, max:2079, min:6


In [ ]:
#Probability of a word appearing in a document in one class , 
#p all items with label ( 0 or 1)
def pr(dtm, y, y_i):
    p = dtm[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
nbratios = np.log(pr(dtm_train, y_train, 1)/pr(dtm_train, y_train, 0))
nbratios = np.squeeze(np.asarray(nbratios))

In [ ]:
print( nbratios[0:10]) 
print( nbratios.shape) 

[ 0.          0.02469261  0.02469261  0.          0.35667494 -0.18232156
  0.08004271  0.01904819 -1.09861229  0.26826399]
(88584,)


In [ ]:
dtm_train[0]

<1x88584 sparse matrix of type '<class 'numpy.int64'>'
	with 315 stored elements in Compressed Sparse Row format>

In [ ]:
def pr_index(dtm, y, y_i,index_key):
    p = dtm[index_key][y[index_key]==y_i].sum(0)
    print(index_key," ) ",p.shape )
    return (p+1) / ((y[index_key]==y_i).sum()+1)

pr_index(dtm_train, y_train, 1,838)    

838  )  (1, 88584)


matrix([[2., 1., 1., ..., 1., 1., 1.]])

In [ ]:
index = 838
dtm = dtm_train[index]
y = y_train[index]
y_i = 1

p = dtm[y==y_i]
p

<1x88584 sparse matrix of type '<class 'numpy.int64'>'
	with 263 stored elements in Compressed Sparse Row format>

In [ ]:
p.sum(0)

matrix([[1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
#indices contributing most to label 1
np.argsort(nbratios)[:-30:-1]

array([  838, 75629, 63102,  1327, 53766, 41455, 55053, 24686, 52431,
       24244, 35408, 65865,  1299, 84021, 46430, 11714, 40076,  1292,
         195,  1230,  2952, 45406, 71926, 10943,  8818, 84915, 84599,
       20743, 42222])

In [ ]:
print("most")
for id in np.argsort(nbratios)[:-20:-1]:
  print("index",id,"  -> ",index_to_word[id], " ratio ", nbratios[id])  
  
print("less")
for id in np.argsort(nbratios)[0:10]:
  print("index",id,"  -> ",index_to_word[id], " ratio ", nbratios[id])    
    

most
index 838   ->  atmosphere  ratio  3.970291913552122
index 75629   ->  «boy  ratio  3.8918202981106265
index 63102   ->  spacecrafts  ratio  3.8501476017100584
index 1327   ->  climax  ratio  3.6888794541139363
index 53766   ->  1871  ratio  3.6888794541139363
index 41455   ->  recipients  ratio  3.637586159726386
index 55053   ->  marshal  ratio  3.6109179126442243
index 24686   ->  fred's  ratio  3.58351893845611
index 52431   ->  dominque  ratio  3.5263605246161616
index 24244   ->  starlift  ratio  3.4657359027997265
index 35408   ->  jaundiced  ratio  3.4011973816621555
index 65865   ->  truce  ratio  3.4011973816621555
index 1299   ->  subtle  ratio  3.4011973816621555
index 84021   ->  screwloose  ratio  3.367295829986474
index 46430   ->  apostle  ratio  3.349904087274605
index 11714   ->  trunk  ratio  3.349904087274605
index 40076   ->  leaks  ratio  3.332204510175204
index 1292   ->  silent  ratio  3.332204510175204
index 195   ->  that's  ratio  3.295836866004329
less

In [ ]:
def get_model(num_words, maxlen, nbratios=None):
    embedding_matrix = np.zeros((num_words, 1))
    for i in range(1, num_words): # skip 0, the padding value
        if nbratios is not None:
            # if log-count ratios are supplied, then it's NBSVM
            embedding_matrix[i] = nbratios[i-1]
        else:
            # if log-count rations are not supplied, this reduces to a logistic regression
            embedding_matrix[i] = 1

    # set up the model
    inp = Input(shape=(maxlen,))
    r = Embedding(num_words, 1, input_length=maxlen, weights=[embedding_matrix], trainable=False)(inp)
    x = Embedding(num_words, 1, input_length=maxlen, embeddings_initializer='glorot_normal')(inp)
    x = dot([r,x], axes=1)
    x = Flatten()(x)
    x = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model

In [ ]:
model = get_model(num_words, maxlen, nbratios=nbratios)
model.fit(x_train, y_train,
          batch_size=32,
          epochs=3,
          validation_data=(x_test, y_test))

Epoch 1/3
782/782 [==============================] - 13s 17ms/step - loss: 0.2691 - accuracy: 0.9434 - val_loss: 0.2504 - val_accuracy: 0.9214
Epoch 2/3
782/782 [==============================] - 13s 16ms/step - loss: 0.0783 - accuracy: 0.9913 - val_loss: 0.2252 - val_accuracy: 0.9240
Epoch 3/3
782/782 [==============================] - 13s 16ms/step - loss: 0.0392 - accuracy: 0.9981 - val_loss: 0.2141 - val_accuracy: 0.9249
